<a href="https://colab.research.google.com/github/xiyuanliang666/NLP/blob/main/NLP_Assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%matplotlib inline

In [ ]:
!pip install torchmetrics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 26.6 MB/s eta 0:00:00


Translation with a Sequence to Sequence Network
*************************************************************

::

    [KEY: > input, = target, < output]

    > il est en train de peindre un tableau .
    = he is painting a picture .
    < he is painting a picture .

    > pourquoi ne pas essayer ce vin delicieux ?
    = why not try that delicious wine ?
    < why not try that delicious wine ?

    > elle n est pas poete mais romanciere .
    = she is not a poet but a novelist .
    < she not not a poet but a novelist .

    > vous etes trop maigre .
    = you re too skinny .
    < you re all alone .

... to varying degrees of success.

This is made possible by the simple but powerful idea of the `sequence
to sequence network <http://arxiv.org/abs/1409.3215>`__, in which two
recurrent neural networks work together to transform one sequence to
another. An encoder network condenses an input sequence into a vector,
and a decoder network unfolds that vector into a new sequence.

.. figure:: /_static/img/seq-seq-images/seq2seq.png
   :alt:

To improve upon this model we'll use an `attention
mechanism <https://arxiv.org/abs/1409.0473>`__, which lets the decoder
learn to focus over a specific range of the input sequence.

**Recommended Reading:**

I assume you have at least installed PyTorch, know Python, and
understand Tensors:

-  http://pytorch.org/ For installation instructions
-  :doc:`/beginner/deep_learning_60min_blitz` to get started with PyTorch in general
-  :doc:`/beginner/pytorch_with_examples` for a wide and deep overview
-  :doc:`/beginner/former_torchies_tutorial` if you are former Lua Torch user


It would also be useful to know about Sequence to Sequence networks and
how they work:

-  `Learning Phrase Representations using RNN Encoder-Decoder for
   Statistical Machine Translation <http://arxiv.org/abs/1406.1078>`__
-  `Sequence to Sequence Learning with Neural
   Networks <http://arxiv.org/abs/1409.3215>`__
-  `Neural Machine Translation by Jointly Learning to Align and
   Translate <https://arxiv.org/abs/1409.0473>`__
-  `A Neural Conversational Model <http://arxiv.org/abs/1506.05869>`__

You will also find the previous tutorials on
:doc:`/intermediate/char_rnn_classification_tutorial`
and :doc:`/intermediate/char_rnn_generation_tutorial`
helpful as those concepts are very similar to the Encoder and Decoder
models, respectively.

And for more, read the papers that introduced these topics:

-  `Learning Phrase Representations using RNN Encoder-Decoder for
   Statistical Machine Translation <http://arxiv.org/abs/1406.1078>`__
-  `Sequence to Sequence Learning with Neural
   Networks <http://arxiv.org/abs/1409.3215>`__
-  `Neural Machine Translation by Jointly Learning to Align and
   Translate <https://arxiv.org/abs/1409.0473>`__
-  `A Neural Conversational Model <http://arxiv.org/abs/1506.05869>`__


**Requirements**



Loading data files
==================

The data for this project is a set of many thousands of English to
French translation pairs.

`This question on Open Data Stack
Exchange <http://opendata.stackexchange.com/questions/3888/dataset-of-sentences-translated-into-many-languages>`__
pointed me to the open translation site http://tatoeba.org/ which has
downloads available at http://tatoeba.org/eng/downloads - and better
yet, someone did the extra work of splitting language pairs into
individual text files here: http://www.manythings.org/anki/

The English to French pairs are too big to include in the repo, so
download to ``data/eng-fra.txt`` before continuing. The file is a tab
separated list of translation pairs:

::

    I am cold.    J'ai froid.

.. Note::
   Download the data from
   `here <https://download.pytorch.org/tutorial/data.zip>`_
   and extract it to the current directory.



In [ ]:
!wget http://www.manythings.org/anki/fra-eng.zip
!unzip -o fra-eng.zip
!mkdir data
!mv fra.txt data/eng-fra.txt

--2025-11-09 02:31:28--  http://www.manythings.org/anki/fra-eng.zip
Resolving www.manythings.org (www.manythings.org)... 173.254.30.110
Connecting to www.manythings.org (www.manythings.org)|173.254.30.110|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8186368 (7.8M) [application/zip]
Saving to: ‘fra-eng.zip’

fra-eng.zip         100%[===================>]   7.81M  36.3MB/s    in 0.2s    

2025-11-09 02:31:28 (36.3 MB/s) - ‘fra-eng.zip’ saved [8186368/8186368]

Archive:  fra-eng.zip
  inflating: _about.txt              
  inflating: fra.txt                 


In [ ]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random
SOS_token = 0
EOS_token = 1


class Lang: # data preprocessing
    def __init__(self, name):
        self.name = name # language name "English" / "French"
        self.word2index = {} # eg. "hello" -> 3
        self.word2count = {} # eg. "hello" -> 5
        self.index2word = {0: "SOS", 1: "EOS"} # initialization: start and end marks
        self.n_words = 2  # Count SOS and EOS, the size of the current vocabulary list

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

The files are all in Unicode, to simplify we will turn Unicode
characters to ASCII, make everything lowercase, and trim most
punctuation.




In [ ]:
# Turn a Unicode string to plain ASCII, thanks to
# http://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s) # transfer Unicode string s to NFD form
        if unicodedata.category(c) != 'Mn' # remove diacritics and other non-ASCII characters
    )

# Lowercase, trim, and remove non-letter characters
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

To read the data file we will split the file into lines, and then split
lines into pairs. The files are all English → Other Language, so if we
want to translate from Other Language → English I added the ``reverse``
flag to reverse the pairs.




In [ ]:
def readLangs(lang1, lang2, reverse=False): # original， tartget，if reverse
    print("Reading lines...")

    # Read the file and split into lines
    lines = open('data/%s-%s.txt' % (lang1, lang2), encoding='utf-8').\
        read().strip().split('\n') # Remove leading and trailing whitespace characters and split by line.

    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')[:2]] for l in lines] # Each row is split by a tab character, containing multiple fields; the first two fields are selected.

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

Since there are a *lot* of example sentences and we want to train
something quickly, we'll trim the data set to only relatively short and
simple sentences. Here the maximum length is 10 words (that includes
ending punctuation) and we're filtering to sentences that translate to
the form "I am" or "He is" etc. (accounting for apostrophes replaced
earlier).




In [ ]:
MAX_LENGTH = 15

eng_prefixes = (
    "i am", "i m",
    "he is", "he s",
    "she is", "she s",
    "you are", "you re",
    "we are", "we re",
    "they are", "they re"
)


def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH and \
        p[1].startswith(eng_prefixes)


def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

The full process for preparing the data is:

-  Read text file and split into lines, split lines into pairs
-  Normalize text, filter by length and content
-  Make word lists from sentences in pairs




In [ ]:
def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs


input_lang, output_lang, pairs = prepareData('eng', 'fra', True)
#print(random.choice(pairs))

Reading lines...
Read 239189 sentence pairs
Trimmed to 23543 sentence pairs
Counting words...
Counted words:
fra 7142
eng 4737


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X = [i[0] for i in pairs]
y = [i[1] for i in pairs]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [ ]:
train_pairs = list(zip(X_train,y_train))
test_pairs = list(zip(X_test,y_test))

The Seq2Seq Model
=================

A Recurrent Neural Network, or RNN, is a network that operates on a
sequence and uses its own output as input for subsequent steps.

A `Sequence to Sequence network <http://arxiv.org/abs/1409.3215>`__, or
seq2seq network, or `Encoder Decoder
network <https://arxiv.org/pdf/1406.1078v3.pdf>`__, is a model
consisting of two RNNs called the encoder and decoder. The encoder reads
an input sequence and outputs a single vector, and the decoder reads
that vector to produce an output sequence.

Unlike sequence prediction with a single RNN, where every input
corresponds to an output, the seq2seq model frees us from sequence
length and order, which makes it ideal for translation between two
languages.

Consider the sentence "Je ne suis pas le chat noir" → "I am not the
black cat". Most of the words in the input sentence have a direct
translation in the output sentence, but are in slightly different
orders, e.g. "chat noir" and "black cat". Because of the "ne/pas"
construction there is also one more word in the input sentence. It would
be difficult to produce a correct translation directly from the sequence
of input words.

With a seq2seq model the encoder creates a single vector which, in the
ideal case, encodes the "meaning" of the input sequence into a single
vector — a single point in some N dimensional space of sentences.




The Encoder
-----------

The encoder of a seq2seq network is a RNN that outputs some value for
every word from the input sentence. For every input word the encoder
outputs a vector and a hidden state, and uses the hidden state for the
next input word.






In [ ]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

The Decoder (Your assignment)
-----------

The decoder is another RNN that takes the encoder output vector(s) and
outputs a sequence of words to create the translation.




Simple Decoder
^^^^^^^^^^^^^^

In the simplest seq2seq decoder we use only last output of the encoder.
This last output is sometimes called the *context vector* as it encodes
context from the entire sequence. This context vector is used as the
initial hidden state of the decoder.

At every step of decoding, the decoder is given an input token and
hidden state. The initial input token is the start-of-string ``<SOS>``
token, and the first hidden state is the context vector (the encoder's
last hidden state).


In [ ]:
class Decoder(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(Decoder, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):

        # Your code here #
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

<div class="alert alert-info"><h4>Note</h4><p>There are other forms of attention that work around the length
  limitation by using a relative position approach. Read about "local
  attention" in `Effective Approaches to Attention-based Neural Machine
  Translation <https://arxiv.org/abs/1508.04025>`__.</p></div>

Training
========

Preparing Training Data
-----------------------

To train, for each pair we will need an input tensor (indexes of the
words in the input sentence) and target tensor (indexes of the words in
the target sentence). While creating these vectors we will append the
EOS token to both sequences.




In [ ]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

Training the Model
------------------

To train we run the input sentence through the encoder, and keep track
of every output and the latest hidden state. Then the decoder is given
the ``<SOS>`` token as its first input, and the last hidden state of the
encoder as its first hidden state.

"Teacher forcing" is the concept of using the real target outputs as
each next input, instead of using the decoder's guess as the next input.
Using teacher forcing causes it to converge faster but `when the trained
network is exploited, it may exhibit
instability <http://minds.jacobs-university.de/sites/default/files/uploads/papers/ESNTutorialRev.pdf>`__.

You can observe outputs of teacher-forced networks that read with
coherent grammar but wander far from the correct translation -
intuitively it has learned to represent the output grammar and can "pick
up" the meaning once the teacher tells it the first few words, but it
has not properly learned how to create the sentence from the translation
in the first place.

Because of the freedom PyTorch's autograd gives us, we can randomly
choose to use teacher forcing or not with a simple if statement. Turn
``teacher_forcing_ratio`` up to use more of it.




In [ ]:
teacher_forcing_ratio = 0.5

def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

This is a helper function to print time elapsed and estimated time
remaining given the current time and progress %.




In [ ]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

The whole training process looks like this:

-  Start a timer
-  Initialize optimizers and criterion
-  Create set of training pairs
-  Start empty losses array for plotting

Then we call ``train`` many times and occasionally print the progress (%
of examples, time so far, estimated time) and average loss.




In [ ]:
def trainIters(encoder, decoder, epochs, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)

    criterion = nn.NLLLoss()

    iter = 1
    n_iters = len(train_pairs) * epochs

    for epoch in range(epochs):
        print("Epoch: %d/%d" % (epoch, epochs))
        for training_pair in train_pairs:
            training_pair = tensorsFromPair(training_pair)

            input_tensor = training_pair[0]
            target_tensor = training_pair[1]

            loss = train(input_tensor, target_tensor, encoder,
                        decoder, encoder_optimizer, decoder_optimizer, criterion)
            print_loss_total += loss
            plot_loss_total += loss

            if iter % print_every == 0:
                print_loss_avg = print_loss_total / print_every
                print_loss_total = 0
                print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                            iter, iter / n_iters * 100, print_loss_avg))

            iter +=1

Evaluation
==========

Evaluation is mostly the same as training, but there are no targets so
we simply feed the decoder's predictions back to itself for each step.
Every time it predicts a word we add it to the output string, and if it
predicts the EOS token we stop there. We also store the decoder's
attention outputs for display later.




In [ ]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []

        for di in range(max_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words

We can evaluate random sentences from the training set and print out the
input, target, and output to make some subjective quality judgements:




In [ ]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [ ]:
from torchmetrics.text.rouge import ROUGEScore
from tqdm import tqdm
import numpy as np

def inference(encoder, decoder, testing_pairs):
    input = []
    gt = []
    predict = []

    from tqdm import tqdm
    for i in tqdm(range(len(testing_pairs))):
        pair = testing_pairs[i]
        output_words = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)

        input.append(pair[0])
        gt.append(pair[1])
        predict.append(output_sentence)

    return input,gt,predict


def eval(gt, predict):
  rouge = ROUGEScore()
  metric_score = rouge(predict, gt)
  print("=== Evaluation score - Rouge score ===")
  print("Rouge1 fmeasure:\t",metric_score["rouge1_fmeasure"].item())
  print("Rouge1 precision:\t",metric_score["rouge1_precision"].item())
  print("Rouge1 recall:  \t",metric_score["rouge1_recall"].item())
  print("Rouge2 fmeasure:\t",metric_score["rouge2_fmeasure"].item())
  print("Rouge2 precision:\t",metric_score["rouge2_precision"].item())
  print("Rouge2 recall:  \t",metric_score["rouge2_recall"].item())
  print("=====================================")

Training and Evaluating
=======================

With all these helper functions in place (it looks like extra work, but
it makes it easier to run multiple experiments) we can actually
initialize a network and start training.

Remember that the input sentences were heavily filtered. For this small
dataset we can use relatively small networks of 256 hidden nodes and a
single GRU layer. After about 40 minutes on a MacBook CPU we'll get some
reasonable results.

.. Note::
   If you run this notebook you can train, interrupt the kernel,
   evaluate, and continue training later. Comment out the lines where the
   encoder and decoder are initialized and run ``trainIters`` again.




In [ ]:
hidden_size = 256
encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
decoder1 = Decoder(hidden_size, output_lang.n_words).to(device)

trainIters(encoder1, decoder1, 5, print_every=5000)

Epoch: 0/5
1m 15s (- 25m 15s) (5000 4%) 3.3327
2m 31s (- 24m 14s) (10000 9%) 2.8679
3m 48s (- 23m 3s) (15000 14%) 2.6160
5m 5s (- 21m 54s) (20000 18%) 2.4385
Epoch: 1/5
6m 23s (- 20m 41s) (25000 23%) 2.2555
7m 41s (- 19m 27s) (30000 28%) 2.0856
8m 58s (- 18m 11s) (35000 33%) 1.9965
10m 16s (- 16m 56s) (40000 37%) 1.8762
Epoch: 2/5
11m 34s (- 15m 40s) (45000 42%) 1.8011
12m 51s (- 14m 23s) (50000 47%) 1.6746
14m 9s (- 13m 6s) (55000 51%) 1.6049
15m 27s (- 11m 50s) (60000 56%) 1.5358
Epoch: 3/5
16m 45s (- 10m 33s) (65000 61%) 1.4951
18m 3s (- 9m 16s) (70000 66%) 1.3860
19m 20s (- 7m 58s) (75000 70%) 1.3431
20m 38s (- 6m 41s) (80000 75%) 1.2745
Epoch: 4/5
21m 56s (- 5m 24s) (85000 80%) 1.2678
23m 15s (- 4m 7s) (90000 84%) 1.1942
24m 32s (- 2m 49s) (95000 89%) 1.1270
25m 50s (- 1m 32s) (100000 94%) 1.0964
27m 8s (- 0m 14s) (105000 99%) 1.0828


In [ ]:
evaluateRandomly(encoder1, decoder1)

> il se gratta la tete .
= he scratched his head .
< he shook his head . <EOS>

> je ne suis pas habituee a faire ca dans l obscurite .
= i m not used to doing that in the dark .
< i m not accustomed to doing this . <EOS>

> je l ai prise pour sa s ur . elles se ressemblent tellement .
= i mistook her for her sister . they look so much alike .
< i mistook her friends so i can help her . <EOS>

> il est celui qui a le plus de chances de reussir .
= he s the most likely to succeed .
< he is the likely to to succeed . <EOS>

> il est a l eglise en ce moment .
= he s at church right now .
< he is now up now . <EOS>

> il me faut y aller seul .
= i must go alone .
< i must go alone . <EOS>

> je suis fauche .
= i m broke .
< i m very . <EOS>

> vous n etes pas censes faire cela n est ce pas ?
= you aren t supposed to do that are you ?
< you aren t supposed to do that are you ? <EOS>

> je dois partir quelque part .
= i must go somewhere .
< i must leave my . <EOS>

> tu es talentueuse .
= y

In [ ]:
# input,gt,predict = inference(encoder1, decoder1, train_pairs)
# eval(gt, predict)

In [ ]:
input,gt,predict = inference(encoder1, decoder1, test_pairs)
eval(gt, predict)

100%|██████████| 2355/2355 [00:12<00:00, 191.47it/s]


=== Evaluation score - Rouge score ===
Rouge1 fmeasure:	 0.6257568597793579
Rouge1 precision:	 0.5845078229904175
Rouge1 recall:  	 0.681381106376648
Rouge2 fmeasure:	 0.4463028013706207
Rouge2 precision:	 0.4093894064426422
Rouge2 recall:  	 0.4985301196575165


Experiment 2: Replace GRU with LSTM in Encoder & Decoder

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# ---------------- EncoderLSTM ----------------
class EncoderLSTM(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.lstm(output, hidden)
        return output, hidden

    def initHidden(self):
        # h_0, c_0 initialize 0
        return (torch.zeros(1, 1, self.hidden_size, device=device),
                torch.zeros(1, 1, self.hidden_size, device=device))

# ---------------- DecoderLSTM ----------------
class DecoderLSTM(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        # hidden-(h, c)
        h, c = hidden
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, (h, c) = self.lstm(output, (h, c))
        output = self.softmax(self.out(output[0]))
        return output, (h, c)

    def initHidden(self):
        return (torch.zeros(1, 1, self.hidden_size, device=device),
                torch.zeros(1, 1, self.hidden_size, device=device))


In [ ]:
hidden_size = 256
encoder_lstm = EncoderLSTM(input_lang.n_words, hidden_size).to(device)
decoder_lstm = DecoderLSTM(hidden_size, output_lang.n_words).to(device)

trainIters(encoder_lstm, decoder_lstm, epochs=5, print_every=5000)

Epoch: 0/5
1m 35s (- 32m 11s) (5000 4%) 3.4031
3m 1s (- 29m 4s) (10000 9%) 3.0374
4m 26s (- 26m 57s) (15000 14%) 2.8388
5m 51s (- 25m 12s) (20000 18%) 2.6760
Epoch: 1/5
7m 17s (- 23m 35s) (25000 23%) 2.4950
8m 51s (- 22m 25s) (30000 28%) 2.3372
10m 27s (- 21m 10s) (35000 33%) 2.2443
11m 53s (- 19m 36s) (40000 37%) 2.1427
Epoch: 2/5
13m 19s (- 18m 2s) (45000 42%) 2.0506
14m 44s (- 16m 29s) (50000 47%) 1.9342
16m 9s (- 14m 58s) (55000 51%) 1.8778
17m 35s (- 13m 28s) (60000 56%) 1.7979
Epoch: 3/5
19m 1s (- 11m 58s) (65000 61%) 1.7612
20m 26s (- 10m 29s) (70000 66%) 1.6671
21m 51s (- 9m 1s) (75000 70%) 1.5957
23m 17s (- 7m 33s) (80000 75%) 1.5432
Epoch: 4/5
24m 43s (- 6m 5s) (85000 80%) 1.5248
26m 8s (- 4m 37s) (90000 84%) 1.4393
27m 34s (- 3m 10s) (95000 89%) 1.3863
28m 59s (- 1m 43s) (100000 94%) 1.3519
30m 25s (- 0m 16s) (105000 99%) 1.3267


In [ ]:
print("\n=== Evaluate Randomly on Training Pairs ===")
evaluateRandomly(encoder_lstm, decoder_lstm)


=== Evaluate Randomly on Training Pairs ===
> ils font de l obstruction pour empecher le projet de loi de passer .
= they are filibustering to prevent the bill from passing .
< they re looking for a the of the the . . . <EOS>

> c est moi qui en suis responsable .
= i am to blame for it .
< i m about to that . <EOS>

> nous sortons pour celebrer .
= we re going out to celebrate .
< we re getting for a . . <EOS>

> je suis un peu au milieu de quelque chose .
= i m kind of in the middle of something .
< i m sort of the something of something . <EOS>

> il est americain .
= he is an american .
< he is poor . <EOS>

> ils sont tous semblables .
= they re all alike .
< they re all alike . <EOS>

> je crains que vous n ayez le mauvais numero .
= i m afraid that you have the wrong number .
< i m afraid you you the wrong number . <EOS>

> j ai hate d aller au zoo avec vous .
= i m looking forward to going to the zoo with you .
< i m looking forward to going to to party you . <EOS>

> il est h

In [ ]:
print("\n=== Evaluate on Test Set (Rouge) ===")
input, gt, predict = inference(encoder_lstm, decoder_lstm, test_pairs)
eval(gt, predict)


=== Evaluate on Test Set (Rouge) ===


100%|██████████| 2355/2355 [00:12<00:00, 181.90it/s]


=== Evaluation score - Rouge score ===
Rouge1 fmeasure:	 0.6129138469696045
Rouge1 precision:	 0.5790438055992126
Rouge1 recall:  	 0.6589285731315613
Rouge2 fmeasure:	 0.4317256808280945
Rouge2 precision:	 0.400726318359375
Rouge2 recall:  	 0.47588860988616943


Experiment 3: Bi-LSTM Encoder + GRU Decoder

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# ---------------- BiLSTM Encoder ----------------
class EncoderBiLSTM(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderBiLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size)
        # Bi LSTM
        self.lstm = nn.LSTM(hidden_size, hidden_size, bidirectional=True)
        # Compress the bidirectional output from 2*hidden_size back to hidden_size, and use it as input to the Decoder.
        self.fc_out = nn.Linear(hidden_size * 2, hidden_size)
        self.fc_h = nn.Linear(hidden_size * 2, hidden_size)
        self.fc_c = nn.Linear(hidden_size * 2, hidden_size)

    def forward(self, input, hidden_tuple):
        if isinstance(hidden_tuple, tuple):
            hidden, cell = hidden_tuple
        else:
            # Pass None on the first call
            hidden = torch.zeros(2, 1, self.hidden_size, device=device)
            cell = torch.zeros(2, 1, self.hidden_size, device=device)
        embedded = self.embedding(input).view(1, 1, -1)
        output, (h, c) = self.lstm(embedded, (hidden, cell))
        # splicing the last hidden state of the forward and reverse directions
        h_cat = torch.cat((h[-2,:,:], h[-1,:,:]), dim=1)  # (1, hidden*2)
        c_cat = torch.cat((c[-2,:,:], c[-1,:,:]), dim=1)
        # Compressed to a unidirectional hidden_size
        h_out = torch.tanh(self.fc_h(h_cat)).unsqueeze(0)
        c_out = torch.tanh(self.fc_c(c_cat)).unsqueeze(0)
        output = self.fc_out(output.squeeze(0)).unsqueeze(0)
        return output, h_out

    def initHidden(self):
        # Bi LSTM → num_layers*2
        return (torch.zeros(2, 1, self.hidden_size, device=device),
                torch.zeros(2, 1, self.hidden_size, device=device))

# ---------------- Decoder (same as baseline GRU) ----------------
class DecoderGRU(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderGRU, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [ ]:
hidden_size = 256
encoder_bilstm = EncoderBiLSTM(input_lang.n_words, hidden_size).to(device)
decoder_gru = DecoderGRU(hidden_size, output_lang.n_words).to(device)

trainIters(encoder_bilstm, decoder_gru, epochs=5, print_every=5000)

Epoch: 0/5
1m 20s (- 27m 15s) (5000 4%) 3.4774
2m 41s (- 25m 49s) (10000 9%) 3.2425
4m 2s (- 24m 29s) (15000 14%) 3.2080
5m 22s (- 23m 7s) (20000 18%) 3.1303
Epoch: 1/5
6m 43s (- 21m 47s) (25000 23%) 3.0782
8m 5s (- 20m 27s) (30000 28%) 3.0512
9m 25s (- 19m 6s) (35000 33%) 3.0567
10m 45s (- 17m 43s) (40000 37%) 2.9774
Epoch: 2/5
12m 5s (- 16m 22s) (45000 42%) 2.9376
13m 29s (- 15m 5s) (50000 47%) 2.9359
14m 50s (- 13m 45s) (55000 51%) 2.9347
16m 11s (- 12m 23s) (60000 56%) 2.9108
Epoch: 3/5
17m 40s (- 11m 7s) (65000 61%) 2.8914
19m 8s (- 9m 49s) (70000 66%) 2.8704
20m 34s (- 8m 29s) (75000 70%) 2.8507
22m 3s (- 7m 9s) (80000 75%) 2.8333
Epoch: 4/5
23m 31s (- 5m 47s) (85000 80%) 2.8369
24m 57s (- 4m 25s) (90000 84%) 2.8094
26m 23s (- 3m 2s) (95000 89%) 2.7959
27m 43s (- 1m 38s) (100000 94%) 2.7794
29m 4s (- 0m 15s) (105000 99%) 2.7636


In [ ]:
print("\n=== Evaluate Randomly on Training Pairs ===")
evaluateRandomly(encoder_bilstm, decoder_gru)


=== Evaluate Randomly on Training Pairs ===
> je suis desole de vous deranger .
= i m sorry to bother you .
< i m not to . . <EOS>

> tu es un petit morveux .
= you re a snotty little brat .
< i m not to . . <EOS>

> vous etes fort ingenieuses .
= you re very clever .
< i m not to . . <EOS>

> je ne vais pas me disputer avec vous !
= i m not going to argue with you .
< i m not to . . <EOS>

> tu es naive .
= you re naive .
< i m not to . . <EOS>

> vous etes aventureuse .
= you re adventurous .
< i m not to . . <EOS>

> vous etes hors de danger .
= you re out of danger .
< i m not to . . <EOS>

> je suis sur qu il reussira l examen .
= i am certain that he will pass the exam .
< i m not to . . <EOS>

> je suis mariee a une avocate .
= i m married to a lawyer .
< i m not to . . <EOS>

> je ne vais pas aller a l ecole aujourd hui .
= i m not going to school today .
< i m not to . . <EOS>



In [ ]:
print("\n=== Evaluate on Test Set (Rouge) ===")
input, gt, predict = inference(encoder_bilstm, decoder_gru, test_pairs)
eval(gt, predict)


=== Evaluate on Test Set (Rouge) ===


100%|██████████| 2355/2355 [00:18<00:00, 130.46it/s]


=== Evaluation score - Rouge score ===
Rouge1 fmeasure:	 0.22234992682933807
Rouge1 precision:	 0.2541826069355011
Rouge1 recall:  	 0.20713137090206146
Rouge2 fmeasure:	 0.10987351834774017
Rouge2 precision:	 0.12590233981609344
Rouge2 recall:  	 0.10480139404535294


修正版本bilstm-encoder

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import time
import math
import random
from torchmetrics.functional.text.rouge import rouge_score

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
MAX_LENGTH = 15
SOS_token = 0
EOS_token = 1
teacher_forcing_ratio = 0.5

# ---------------- BiLSTM Encoder (match GRU Decoder) ----------------
class EncoderBiLSTM(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderBiLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size, num_layers=1, bidirectional=True)
        self.fc_h = nn.Linear(hidden_size * 2, hidden_size)

    def forward(self, input, hidden_tuple):
        embedded = self.embedding(input).view(1, 1, -1)
        output, (h_out, c_out) = self.lstm(embedded, hidden_tuple)
        return output, (h_out, c_out)

    def initHidden(self):
        return (torch.zeros(2, 1, self.hidden_size, device=device),
                torch.zeros(2, 1, self.hidden_size, device=device))

    def get_decoder_hidden(self, h_final):
        # h_final: [2, 1, hidden_size]
        h_cat = torch.cat((h_final[-2,:,:], h_final[-1,:,:]), dim=1)  # [1, 2 * hidden_size]
        decoder_hidden = torch.tanh(self.fc_h(h_cat)).unsqueeze(0)
        return decoder_hidden

# ---------------- Decoder GRU ----------------
class DecoderGRU(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderGRU, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

# ---------------- train function (match Bi-LSTM) ----------------
def train(input_tensor, target_tensor, encoder, decoder,
          encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):

    # EncoderBiLSTM return (h, c) tuple
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    # GRU Decoder don't need encoder_outputs. Therefore, the encoder_outputs tensor does not need to be defined here.

    loss = 0

    # 1. Encoder loop：Correctly transmits (h, c) tuples
    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
        # encoder_hidden here is the updated (h, c) tuple.

    # 2. State compression: Transforming the final state of the Bi-LSTM into the initial state of the GRU.
    # encoder_hidden is (h_final, c_final)
    h_final = encoder_hidden[0] # get h state [2, 1, hidden_size]
    decoder_hidden = encoder.get_decoder_hidden(h_final) # use method in Encoder to compress and transfer

    decoder_input = torch.tensor([[SOS_token]], device=device)

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing
    else:
        # Without teacher forcing: use its own prediction as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # Detach from history as input
            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()
    encoder_optimizer.step()
    decoder_optimizer.step()
    return loss.item() / target_length

# ---------------- evaluate function (match Bi-LSTM) ----------------
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    """
    Evaluates the Seq2Seq model on a single sentence.
    This version is fixed to handle the (h, c) tuple of the Bi-LSTM Encoder
    and the tuple returned by tensorsFromPair.
    """
    with torch.no_grad():
        input_tensor, _ = tensorsFromPair([sentence, ''])
        input_length = input_tensor.size()[0]

        encoder_hidden = encoder.initHidden()
        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)

        h_final = encoder_hidden[0]
        decoder_hidden = encoder.get_decoder_hidden(h_final)

        decoder_input = torch.tensor([[SOS_token]], device=device)

        decoded_words = []
        for di in range(max_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            topv, topi = decoder_output.data.topk(1)

            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words

In [ ]:
hidden_size = 256
encoder_bilstm = EncoderBiLSTM(input_lang.n_words, hidden_size).to(device)
decoder_gru = DecoderGRU(hidden_size, output_lang.n_words).to(device)
trainIters(encoder_bilstm, decoder_gru, epochs=5, print_every=5000)


Epoch: 0/5
1m 39s (- 33m 24s) (5000 4%) 3.4484
3m 18s (- 31m 42s) (10000 9%) 3.0274
4m 58s (- 30m 6s) (15000 14%) 2.8729
6m 39s (- 28m 36s) (20000 18%) 2.6978
Epoch: 1/5
8m 21s (- 27m 4s) (25000 23%) 2.4994
10m 3s (- 25m 26s) (30000 28%) 2.3265
11m 45s (- 23m 50s) (35000 33%) 2.2179
13m 27s (- 22m 11s) (40000 37%) 2.1039
Epoch: 2/5
15m 10s (- 20m 33s) (45000 42%) 1.9947
16m 53s (- 18m 53s) (50000 47%) 1.8672
18m 35s (- 17m 13s) (55000 51%) 1.7846
20m 17s (- 15m 32s) (60000 56%) 1.7081
Epoch: 3/5
22m 0s (- 13m 51s) (65000 61%) 1.6507
23m 42s (- 12m 10s) (70000 66%) 1.5378
25m 24s (- 10m 28s) (75000 70%) 1.4950
27m 7s (- 8m 47s) (80000 75%) 1.4155
Epoch: 4/5
28m 50s (- 7m 6s) (85000 80%) 1.4132
30m 33s (- 5m 24s) (90000 84%) 1.3226
32m 16s (- 3m 42s) (95000 89%) 1.2513
33m 57s (- 2m 1s) (100000 94%) 1.2288
35m 40s (- 0m 19s) (105000 99%) 1.1969


In [ ]:

print("\n=== Evaluate Randomly on Training Pairs ===")
evaluateRandomly(encoder_bilstm, decoder_gru)


=== Evaluate Randomly on Training Pairs ===
> vous etes pleine de surprises .
= you re full of surprises .
< you re so talented . <EOS>

> je vous ai peut etre deja raconte cette histoire .
= i may have told you this story before .
< i may have you before i ve made you . <EOS>

> je suis heureuse pour vous deux .
= i m happy for you both .
< i m happy for you . <EOS>

> elle parle chinois .
= she speaks chinese .
< she speaks chinese . <EOS>

> on dirait qu il est riche .
= he seems to be rich .
< he seems to be rich now . <EOS>

> j essaie de proteger tom .
= i m trying to protect tom .
< i m trying to protect tom . <EOS>

> je ne suis pas du tout convaincu .
= i m not convinced at all .
< i m not a at all <EOS>

> il ne m a jamais frappe auparavant .
= he s never hit me before .
< he sent me before before . <EOS>

> il est populaire aupres de tout le monde .
= he is popular with everybody .
< he is popular with everybody . <EOS>

> vous etes la femme la plus belle au monde .
= you r

In [ ]:
print("\n=== Evaluate on Test Set (Rouge) ===")
input, gt, predict = inference(encoder_bilstm, decoder_gru, test_pairs)
eval(gt, predict)


=== Evaluate on Test Set (Rouge) ===


100%|██████████| 2355/2355 [00:13<00:00, 172.61it/s]


=== Evaluation score - Rouge score ===
Rouge1 fmeasure:	 0.6115792393684387
Rouge1 precision:	 0.5845984220504761
Rouge1 recall:  	 0.6505200862884521
Rouge2 fmeasure:	 0.42236021161079407
Rouge2 precision:	 0.3969039022922516
Rouge2 recall:  	 0.46048763394355774


Experiment 4: Add Attention between Encoder and Decoder (Bahdanau)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
MAX_LENGTH = 15
SOS_token = 0
EOS_token = 1
teacher_forcing_ratio = 0.5

# ---- Attention ----
class BahdanauAttention(nn.Module):
    def __init__(self, hidden_size):
        super(BahdanauAttention, self).__init__()
        self.hidden_size = hidden_size
        self.attn = nn.Linear(hidden_size * 2, hidden_size)
        self.v = nn.Parameter(torch.rand(hidden_size))

    def forward(self, hidden, encoder_outputs):
        max_len = encoder_outputs.size(0)
        hidden = hidden.repeat(max_len, 1, 1).transpose(0, 1)
        energy = torch.tanh(self.attn(torch.cat((hidden, encoder_outputs.unsqueeze(0)), 2)))
        energy = energy.squeeze(0)
        energy = torch.matmul(energy, self.v)
        attn_weights = F.softmax(energy, dim=0)
        return attn_weights.unsqueeze(0)

# ---- Encoder ----
class EncoderRNN_Attn(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN_Attn, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output, hidden = self.gru(embedded, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

# ---- Decoder with Attention（include buffer, only set once） ----
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.max_length = max_length

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.attention = BahdanauAttention(hidden_size)
        self.gru = nn.GRU(hidden_size * 2, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)
        self._encoder_outputs_buffer = None

    def set_encoder_outputs(self, encoder_outputs):
        self._encoder_outputs_buffer = encoder_outputs

    def forward(self, input, hidden):
        encoder_outputs = self._encoder_outputs_buffer
        if encoder_outputs is None:
            raise RuntimeError("Must call set_encoder_outputs before decoding.")
        embedded = self.embedding(input).view(1, 1, -1)
        attn_weights = self.attention(hidden, encoder_outputs)
        context = torch.bmm(attn_weights.unsqueeze(0), encoder_outputs.unsqueeze(0))
        rnn_input = torch.cat((embedded, context), dim=2)
        output, hidden = self.gru(rnn_input, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

# ---- train ----
def train_with_attention(input_tensor, target_tensor, encoder, decoder,
                         encoder_optimizer, decoder_optimizer, criterion,
                         max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)
    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)
    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]
    decoder.set_encoder_outputs(encoder_outputs)

    decoder_input = torch.tensor([[SOS_token]], device=device)
    decoder_hidden = encoder_hidden
    use_teacher_forcing = True if torch.rand(1).item() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        for di in range(target_length):
            decoder_output, decoder_hidden, _ = decoder(decoder_input, decoder_hidden)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]
    else:
        for di in range(target_length):
            decoder_output, decoder_hidden, _ = decoder(decoder_input, decoder_hidden)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()
            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()
    encoder_optimizer.step()
    decoder_optimizer.step()
    return loss.item() / target_length

def trainIters_with_attention(encoder, decoder, epochs, print_every=1000, learning_rate=0.01):
    import time, math
    def asMinutes(s):
        m = math.floor(s / 60)
        s -= m * 60
        return '%dm %ds' % (m, s)
    def timeSince(since, percent):
        now = time.time()
        s = now - since
        es = s / (percent)
        rs = es - s
        return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

    start = time.time()
    print_loss_total = 0
    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    criterion = nn.NLLLoss()
    n_iters = len(train_pairs) * epochs
    it = 1
    for epoch in range(epochs):
        print(f"\nEpoch: {epoch+1}/{epochs}")
        for training_pair in train_pairs:
            input_tensor, target_tensor = tensorsFromPair(training_pair)
            loss = train_with_attention(input_tensor, target_tensor, encoder, decoder,
                                        encoder_optimizer, decoder_optimizer, criterion)
            print_loss_total += loss
            if it % print_every == 0:
                print_loss_avg = print_loss_total / print_every
                print_loss_total = 0
                print(f"{timeSince(start, it / n_iters)} ({it}/{n_iters}) Loss={print_loss_avg:.4f}")
            it += 1

# ---- evaluate include buffer setting（method 1） ----
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()
        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)
        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]
        # directly setting buffer in evaluate
        decoder.set_encoder_outputs(encoder_outputs)

        decoder_input = torch.tensor([[SOS_token]], device=device)
        decoder_hidden = encoder_hidden
        decoded_words = []

        for di in range(max_length):
            decoder_output, decoder_hidden, _ = decoder(decoder_input, decoder_hidden)   # unpack为3
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])
            decoder_input = topi.squeeze().detach()

        return decoded_words

# ---- Compatible with original evaluateRandomly ----
def evaluateRandomly_with_attn(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

# ---- Compatible with original inference ----
def inference_with_attn(encoder, decoder, testing_pairs):
    input_l, gt_l, predict_l = [], [], []
    from tqdm import tqdm
    for pair in tqdm(testing_pairs):
        output_words = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        input_l.append(pair[0])
        gt_l.append(pair[1])
        predict_l.append(output_sentence)
    return input_l, gt_l, predict_l



In [ ]:
hidden_size = 256
encoder_attn = EncoderRNN_Attn(input_lang.n_words, hidden_size).to(device)
decoder_attn = AttnDecoderRNN(hidden_size, output_lang.n_words).to(device)
trainIters_with_attention(encoder_attn, decoder_attn, epochs=5, print_every=5000)


Epoch: 1/5
1m 47s (- 36m 4s) (5000/105940) Loss=3.2955
3m 35s (- 34m 24s) (10000/105940) Loss=2.7590
5m 24s (- 32m 45s) (15000/105940) Loss=2.5110
7m 13s (- 31m 0s) (20000/105940) Loss=2.3154

Epoch: 2/5
9m 3s (- 29m 18s) (25000/105940) Loss=2.1506
10m 52s (- 27m 31s) (30000/105940) Loss=1.9978
12m 42s (- 25m 45s) (35000/105940) Loss=1.8849
14m 32s (- 23m 58s) (40000/105940) Loss=1.7942

Epoch: 3/5
16m 22s (- 22m 10s) (45000/105940) Loss=1.6942
18m 13s (- 20m 23s) (50000/105940) Loss=1.5920
20m 5s (- 18m 36s) (55000/105940) Loss=1.5239
21m 57s (- 16m 48s) (60000/105940) Loss=1.4559

Epoch: 4/5
23m 50s (- 15m 0s) (65000/105940) Loss=1.4236
25m 42s (- 13m 12s) (70000/105940) Loss=1.3360
27m 35s (- 11m 22s) (75000/105940) Loss=1.2721
29m 27s (- 9m 33s) (80000/105940) Loss=1.1953

Epoch: 5/5
31m 20s (- 7m 43s) (85000/105940) Loss=1.2236
33m 12s (- 5m 52s) (90000/105940) Loss=1.1624
35m 4s (- 4m 2s) (95000/105940) Loss=1.1040
36m 56s (- 2m 11s) (100000/105940) Loss=1.0687
38m 49s (- 0m 20s

In [ ]:
print("\n=== Evaluate Randomly on Training Pairs ===")
evaluateRandomly_with_attn(encoder_attn, decoder_attn)



=== Evaluate Randomly on Training Pairs ===
> je suis desolee mais j ai deja une petite copine .
= i m sorry but i already have a girlfriend .
< i m sorry but i already have a girlfriend . <EOS>

> nous ne sommes pas toutes aussi fatiguees que vous .
= we re not all as tired as you are .
< we re not all as tired as you you you are . <EOS>

> je ne vous voulais aucun mal .
= i meant you no harm .
< i meant you to hurt . <EOS>

> je plaisante .
= i m just joking .
< i m not kidding . <EOS>

> je suis desole nous avons ete devalises .
= i m sorry we re completely sold out .
< i m sorry i we ve been been . <EOS>

> je suis timide .
= i m shy .
< i m shy . <EOS>

> c est une connaissance de ma femme .
= he is acquainted with my wife .
< he is my wife . <EOS>

> je suis un etudiant de cette ecole .
= i am a student of this school .
< i am a student of this school . <EOS>

> j ai un peu sommeil .
= i m feeling kind of sleepy .
< i m feeling sleepy sleepy . <EOS>

> je ne suis pas malheureux 

In [ ]:
print("\n=== Evaluate on Test Set (Rouge) ===")
input, gt, predict = inference_with_attn(encoder_attn, decoder_attn, test_pairs)
eval(gt, predict)


=== Evaluate on Test Set (Rouge) ===


100%|██████████| 2355/2355 [00:17<00:00, 133.99it/s]


=== Evaluation score - Rouge score ===
Rouge1 fmeasure:	 0.6334235072135925
Rouge1 precision:	 0.5939782857894897
Rouge1 recall:  	 0.6873691082000732
Rouge2 fmeasure:	 0.4555526673793793
Rouge2 precision:	 0.42015090584754944
Rouge2 recall:  	 0.5062500834465027


Experiment 5: Transformer Encoder + GRU Decoder

In [ ]:
import math  # positional encoding

# -------------------------- PositionalEncoding--------------------------
class PositionalEncoding(nn.Module):
    def __init__(self, dim_model, dropout_p, max_len=5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout_p)
        pe = torch.zeros(max_len, dim_model, device=device)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, dim_model, 2).float() * (-math.log(10000.0) / dim_model))

        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pos_encoding', pe)

    def forward(self, x):
        # x：[batch_size, seq_len, dim_model] 或 [seq_len, batch_size, dim_model]
        seq_len = x.size(1) if x.dim() == 3 else x.size(0)
        x = x + self.pos_encoding[:, :seq_len, :]
        return self.dropout(x)

class TransformerEncoder(nn.Module):
    def __init__(self, input_size, hidden_size, nhead=2, num_layers=2, dropout_p=0.1):
        super(TransformerEncoder, self).__init__()
        self.hidden_size = hidden_size  # same as Decoder hidden
        self.dim_model = hidden_size    # Transformer input/output dimensions (aligned with hidden layer dimensions)


        self.embedding = nn.Embedding(input_size, self.dim_model, device=device)
        self.pos_encoder = PositionalEncoding(
            dim_model=self.dim_model,
            dropout_p=dropout_p,
            max_len=MAX_LENGTH
        )

        encoder_layers = nn.TransformerEncoderLayer(
            d_model=self.dim_model,
            nhead=nhead,
            dim_feedforward=self.dim_model * 4,
            dropout=dropout_p,
            batch_first=True,  # [batch_size, seq_len, dim_model]
            activation='relu'
        )
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, num_layers=num_layers)
        self.fc = nn.Linear(self.dim_model, self.hidden_size, device=device)

    def forward(self, input, hidden=None):
        # Adapts to one-dimensional input (during evaluation) and two-dimensional input (during training).
        if input.dim() == 1:
            # one-dimensional：[seq_len] → [1, seq_len]（batch_size=1）
            input_seq = input.unsqueeze(0)
        else:
            # two-dimensional：[seq_len, 1] → [1, seq_len]（batch_size=1）
            input_seq = input.squeeze(1).unsqueeze(0)

        # input：[seq_len, 1]（original GRU input form）→ [1, seq_len]（batch_first=True）
        # To meet the task requirements: input the entire sentence at once, rather than feeding it in word by word.
        # input_seq = input.squeeze(1).unsqueeze(0)  # [batch_size=1, seq_len]


        embedded = self.embedding(input_seq) * math.sqrt(torch.tensor(self.dim_model, device=device))

        embedded = self.pos_encoder(embedded)  # [1, seq_len, dim_model]


        transformer_out = self.transformer_encoder(embedded)  # [1, seq_len, dim_model]

        # Generate sentence representations (core task requirement: take the average of all hidden representations of tokens).
        sentence_embedding = transformer_out.mean(dim=1)  # [1, dim_model](Average value based on seq_len dimension)
        sentence_embedding = self.fc(sentence_embedding).unsqueeze(0)  # [1, 1, hidden_size](Match the Decoder input shape)

        encoder_outputs = self.fc(transformer_out.squeeze(0))  # [seq_len, hidden_size]

        encoder_outputs_padded = torch.zeros(MAX_LENGTH, self.hidden_size, device=device)
        encoder_outputs_padded[:encoder_outputs.size(0), :] = encoder_outputs

        return encoder_outputs_padded, sentence_embedding

    def initHidden(self):
        # To maintain consistency with the original interface, the actual Transformer does not require an initial hidden state.
        return torch.zeros(1, 1, self.hidden_size, device=device)

class DecoderGRU(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderGRU, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(output_size, hidden_size, device=device)
        self.gru = nn.GRU(hidden_size, hidden_size, device=device)
        self.out = nn.Linear(hidden_size, output_size, device=device)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

# -------------------------- train（match Transformer）--------------------------
def train_transformer(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion):
    encoder_hidden = encoder.initHidden()
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)
    loss = 0

    # Transformer encodes the entire sequence at once (replacing the original GRU loop).
    encoder_outputs, encoder_hidden = encoder(input_tensor, encoder_hidden)

    decoder_input = torch.tensor([[SOS_token]], device=device)
    decoder_hidden = encoder_hidden
    use_teacher_forcing = random.random() < teacher_forcing_ratio

    if use_teacher_forcing:
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]
    else:
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()
            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()
    encoder_optimizer.step()
    decoder_optimizer.step()
    return loss.item() / target_length

def trainIters_transformer(encoder, decoder, epochs, print_every=1000, learning_rate=0.01):
    start = time.time()
    print_loss_total = 0
    iter = 1
    n_iters = len(train_pairs) * epochs

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    criterion = nn.NLLLoss()

    for epoch in range(epochs):
        print(f"\nEpoch: {epoch+1}/{epochs}")
        for training_pair in train_pairs:
            training_pair = tensorsFromPair(training_pair)
            input_tensor = training_pair[0]
            target_tensor = training_pair[1]

            loss = train_transformer(input_tensor, target_tensor, encoder,
                                     decoder, encoder_optimizer, decoder_optimizer, criterion)
            print_loss_total += loss

            if iter % print_every == 0:
                print_loss_avg = print_loss_total / print_every
                print_loss_total = 0
                print(f'{timeSince(start, iter / n_iters)} ({iter} {iter/n_iters*100:.0f}%) Loss: {print_loss_avg:.4f}')

            iter += 1

In [ ]:
# -------------------------- Model initialization and training evaluation --------------------------
# Model parameters (consistent with the original GRU model to ensure fairness)
hidden_size = 256
encoder_transformer = TransformerEncoder(
    input_size=input_lang.n_words,
    hidden_size=hidden_size,
    nhead=2,  # Number of attention heads (controlling model complexity)
    num_layers=2  # The number of Transformer layers is comparable to the complexity of a single-layer GRU.
).to(device)
decoder_gru = DecoderGRU(hidden_size, output_lang.n_words).to(device)

trainIters_transformer(encoder_transformer, decoder_gru, epochs=5, print_every=5000, learning_rate=0.001)


Epoch: 1/5
1m 11s (- 23m 59s) (5000 5%) Loss: 3.5460
2m 20s (- 22m 29s) (10000 9%) Loss: 3.1716
3m 29s (- 21m 11s) (15000 14%) Loss: 3.0990
4m 39s (- 20m 3s) (20000 19%) Loss: 3.0059

Epoch: 2/5
5m 50s (- 18m 54s) (25000 24%) Loss: 2.9044
7m 0s (- 17m 45s) (30000 28%) Loss: 2.8013
8m 11s (- 16m 36s) (35000 33%) Loss: 2.7509
9m 22s (- 15m 26s) (40000 38%) Loss: 2.6784

Epoch: 3/5
10m 32s (- 14m 16s) (45000 42%) Loss: 2.6271
11m 42s (- 13m 6s) (50000 47%) Loss: 2.5309
12m 53s (- 11m 56s) (55000 52%) Loss: 2.4872
14m 3s (- 10m 45s) (60000 57%) Loss: 2.4351

Epoch: 4/5
15m 14s (- 9m 35s) (65000 61%) Loss: 2.4240
16m 24s (- 8m 25s) (70000 66%) Loss: 2.3335
17m 35s (- 7m 15s) (75000 71%) Loss: 2.2818
18m 45s (- 6m 5s) (80000 76%) Loss: 2.2528

Epoch: 5/5
19m 56s (- 4m 54s) (85000 80%) Loss: 2.2542
21m 6s (- 3m 44s) (90000 85%) Loss: 2.1549
22m 16s (- 2m 33s) (95000 90%) Loss: 2.1156
23m 27s (- 1m 23s) (100000 94%) Loss: 2.1009
24m 38s (- 0m 13s) (105000 99%) Loss: 2.0855


In [ ]:
print("\n=== Evaluate Randomly on Training Pairs ===")
evaluateRandomly(encoder_transformer, decoder_gru)


=== Evaluate Randomly on Training Pairs ===
> je ne me sens pas tres bien .
= i m not feeling so good .
< you re famous . <EOS>

> je suis accro a la nicotine .
= i m addicted to nicotine .
< you re early . <EOS>

> tu n es plus seul .
= you re not alone anymore .
< you re off . <EOS>

> tu es grognon .
= you re grumpy .
< you re up . <EOS>

> je dois terminer mes devoirs .
= i must get my homework finished .
< you re famous . <EOS>

> je suis trop fatiguee pour me disputer .
= i m too tired to argue .
< you re off . <EOS>

> je le verrai demain .
= i m going to see him tomorrow .
< you re famous . <EOS>

> tu n es pas en colere contre moi n est ce pas ?
= you re not mad at me are you ?
< you re off . <EOS>

> il est force de remarquer votre erreur .
= he s bound to notice your mistake .
< you re a . <EOS>

> je suis tout seul .
= i m all by myself .
< you re off . <EOS>



In [ ]:
print("\n=== Evaluate on Test Set (Rouge) ===")
input, gt, predict = inference(encoder_transformer, decoder_gru, test_pairs)
eval(gt, predict)


=== Evaluate on Test Set (Rouge) ===


100%|██████████| 2355/2355 [00:34<00:00, 69.06it/s]


=== Evaluation score - Rouge score ===
Rouge1 fmeasure:	 0.13281027972698212
Rouge1 precision:	 0.15431706607341766
Rouge1 recall:  	 0.12352804094552994
Rouge2 fmeasure:	 0.053510479629039764
Rouge2 precision:	 0.06185421347618103
Rouge2 recall:  	 0.051382262259721756


In [ ]:
hidden_size = 256
encoder_transformer = TransformerEncoder(
    input_size=input_lang.n_words,
    hidden_size=hidden_size,
    nhead=2,
    num_layers=2
).to(device)
decoder_gru = DecoderGRU(hidden_size, output_lang.n_words).to(device)

# epoch=20
trainIters_transformer(encoder_transformer, decoder_gru, epochs=20, print_every=5000, learning_rate=0.001)


Epoch: 1/20
1m 5s (- 91m 59s) (5000 1%) Loss: 3.5291
2m 14s (- 92m 48s) (10000 2%) Loss: 3.1865
3m 25s (- 93m 6s) (15000 4%) Loss: 3.1044
4m 35s (- 92m 38s) (20000 5%) Loss: 3.0125

Epoch: 2/20
5m 47s (- 92m 18s) (25000 6%) Loss: 2.8977
6m 58s (- 91m 27s) (30000 7%) Loss: 2.8104
8m 9s (- 90m 33s) (35000 8%) Loss: 2.7617
9m 20s (- 89m 37s) (40000 9%) Loss: 2.6884

Epoch: 3/20
10m 32s (- 88m 39s) (45000 11%) Loss: 2.6341
11m 43s (- 87m 35s) (50000 12%) Loss: 2.5447
12m 54s (- 86m 31s) (55000 13%) Loss: 2.5135
14m 5s (- 85m 26s) (60000 14%) Loss: 2.4461

Epoch: 4/20
15m 17s (- 84m 22s) (65000 15%) Loss: 2.4281
16m 28s (- 83m 14s) (70000 17%) Loss: 2.3305
17m 39s (- 82m 6s) (75000 18%) Loss: 2.3017
18m 50s (- 80m 58s) (80000 19%) Loss: 2.2493

Epoch: 5/20
20m 2s (- 79m 50s) (85000 20%) Loss: 2.2557
21m 13s (- 78m 42s) (90000 21%) Loss: 2.1749
22m 24s (- 77m 31s) (95000 22%) Loss: 2.1377
23m 35s (- 76m 22s) (100000 24%) Loss: 2.1165
24m 46s (- 75m 13s) (105000 25%) Loss: 2.0955

Epoch: 6/2

In [ ]:
print("\n=== Evaluate Randomly on Training Pairs ===")
evaluateRandomly(encoder_transformer, decoder_gru)


=== Evaluate Randomly on Training Pairs ===
> il commande rarement quoi que ce soit de nouveau .
= he seldom orders anything new .
< they . <EOS>

> je suis maintenant habituee au port du masque .
= i m now used to wearing a mask .
< they . <EOS>

> nous envisageons l achat de nouveaux meubles .
= we re thinking of buying some new furniture .
< they re out . <EOS>

> je quitte l australie .
= i m leaving australia .
< they . <EOS>

> il n est plus ce qu il etait .
= he is not what he was .
< <EOS>

> j ai du mal a me concentrer .
= i m having trouble focusing .
< they re up . <EOS>

> tu n es pas non plus un saint .
= you re not a saint either .
< they . <EOS>

> il est tres peu probable que je sois licencie pour avoir fait ca .
= i m not very likely to be fired for doing that .
< <EOS>

> je suis toujours votre ami .
= i m still your friend .
< they . <EOS>

> vous etes les etudiantes de tom n est ce pas ?
= you re tom s students right ?
< they . <EOS>



In [ ]:
print("\n=== Evaluate on Test Set (Rouge) ===")
input, gt, predict = inference(encoder_transformer, decoder_gru, test_pairs)
eval(gt, predict)


=== Evaluate on Test Set (Rouge) ===


100%|██████████| 2355/2355 [00:32<00:00, 71.72it/s]


=== Evaluation score - Rouge score ===
Rouge1 fmeasure:	 0.02623540721833706
Rouge1 precision:	 0.03557678684592247
Rouge1 recall:  	 0.022507069632411003
Rouge2 fmeasure:	 0.00218442571349442
Rouge2 precision:	 0.0026185421738773584
Rouge2 recall:  	 0.0021834815852344036
